# Функционално моделиране: Aлгебрични Tипове Данни

*Д-р Александър Александров (alexandroval@vmware.com), лекция пред ученици от НПМГ, 2020/05/11*

Най-бързият начин за тази презентация, може да отворите нов онлайн едитор в https://scastie.scala-lang.org/.

## Характеристики на функционално програмиране 

От синтактична гледна точка, основната характеристика един функционален език за програмиране е възможността функциите да бъдат третирани като стойности, т.е. да могат да бъдат асоциирани със имволи и да бъдат използвани като аргументи и параметри на други (т.нар. higher-order) функции.

In [1]:
val f = (x: Int) => x + 42 // regular function bound to a symbol x
val g = (h: Int => Int) => ((x: Int) => h(x) - 15) // higher-order function g with a function parameter f

f(0) // evaulate f at point 0
g(f) // evaluate g at f, the result i another function that maps integers to integers

f: Int => Int = ammonite.$sess.cmd0$Helper$$Lambda$2247/0x0000000800beb840@548cd963
g: Int => Int => Int => Int = ammonite.$sess.cmd0$Helper$$Lambda$2248/0x0000000800bec840@23be8b2
res0_2: Int = 42
res0_3: Int => Int = ammonite.$sess.cmd0$Helper$$Lambda$2249/0x0000000800bed040@674932c7

От семантична гледа точка, есенцията на функционалното функционалното програмирне се крие в това, че когато един символ се дефинира, стойността, която му е зададенана, не може да се променя впоследствие (т.нар. immutability).

In [2]:
val x = 5

x: Int = 5

In [2]:
x = 6 // this is going to throw an error

cmd2.sc:1: reassignment to val
val res2 = x = 6 // this is going to throw an error
             ^Compilation Failed

: 

Най-общо казано, в резултат на това свойство можем да сме сигурни, че всеки един програмен израз обозначава конкретна стойност, и още повече, това значение зависи само от самия програмен код, а не от някакъв "невидим" контекст. Това свойство на функционалните програми се нарича референциална шрозрачност (referntial transparency).

Напеример, въпреки че не са **идентични като код**, функциите `f` и `g` са **еквивалентни**, тъй като равенството `f(x,y) == g(x,y)` е изъплнено за всяка комбинация от числа `x` и `y`.

In [3]:
val f = (x: Int, y: Int) => x + y
val g = (x: Int, y: Int) => y + x

f: (Int, Int) => Int = ammonite.$sess.cmd2$Helper$$Lambda$2365/0x0000000800c4d040@486f824e
g: (Int, Int) => Int = ammonite.$sess.cmd2$Helper$$Lambda$2366/0x0000000800c4e040@5f5967cf

За да се уверим в горното твърдение не е нужно да емпирично да изпълняваме програмата `f(x,y) == g(x,y)` за всяка възможна комбинация от стойности `x: Int` и `y: Int`.

In [4]:
val random = new scala.util.Random()

random: scala.util.Random = scala.util.Random@2c5876d0

In [5]:
// repeat the execution of thi cell in order to get empirical data on the validity of the proposition 
// "f(x,y) == g(x,y)" 
val x = random.nextInt(1000)
val y = random.nextInt(1000)
f(x, y) == g(y, x)

x: Int = 910
y: Int = 727
res4_2: Boolean = true


На база на (функционалния) код дефиниран по-горе можем да заключим горното твърдение и аналитично, вследствие на факта, че 

```
f(x, y) == x + y == y + x = g(x, y)
```

тъй като събирането е комутативно.

## Алгебрични типове данни

Алгебричните типове данни биват два вида - типове-произведения и типове-суми.

Абстрактно погледнато (т.е., независимо от конкретния език за програмиране), всеки тип може да бъде описан и като сума от произведения. 

Следващите секции въвеждат тези понятия на базата на конкретни примери.

### Типове-Произведения (Product Types)

В Scala, типовете-произведения се дефинират с помощта на ключовата дума `case class`. Например, :

In [6]:
case class HighSchoolStudent(socialID: Int, name: String, surname: String)
case class UniversityStudent(first: String, last: String, studentID: Int)

defined class HighSchoolStudent
defined class UniversityStudent

Обърнете внимание, че и двата типа комбинират по две стойности от тип `String` с една от тип `Int`.
Абстрактното погледнато, двата типа репрезентират конкретни произведения на типове (product type space) `T1 × ... × Tn`:

- `HighSchoolStudent(socialID: Int, name: String, surname: String) ≅ Int × String × String` (в първия случай) и 
- `UniversityStudent(first: String, last: String, studentID: Int) ≅ String × String × Int` (във втория).

Типовите произведения от дясната страна генерализират номиналнте дефиниции от лявата до най-чистата им структура. Образно казано, забравили сме името на самия тип (`HighSchoolStudent`/`UniversityStudent`) и на конкретните параметри (`name`/`last`, `surname`/`last` и `socialID`/`studentID` ).

Тъй като тип-конструктора за умножение `·×·` е комутативен и асоциативен, типовете `UniversityStudent` и `HighSchoolStudent` са изоморфни помежду си - т.е., на всеки студент `x: UniversityStudent` съответства точно един гимназист `y: HighSchoolStudent` и обратно.

```
HighSchoolStudent(...) ≅ Int × String × String ≅ String × String × Int ≅ UniversityStudent(...)
```

Доказателство за горното твърдение е съществуването на следните две функции 

- `hsToUni: HighSchoolStudent => UniversityStudent` и 
- `uniToHs: UniversityStudent => HighSchoolStudent`.

In [7]:
val hsToUni = (x: HighSchoolStudent) => UniversityStudent(x.name, x.surname, x.socialID)
val uniToHS = (x: UniversityStudent) => HighSchoolStudent(x.studentID, x.first, x.last)

hsToUni: HighSchoolStudent => UniversityStudent = ammonite.$sess.cmd6$Helper$$Lambda$2592/0x0000000800cfa040@8970b7
uniToHS: UniversityStudent => HighSchoolStudent = ammonite.$sess.cmd6$Helper$$Lambda$2593/0x0000000800cfa840@48b9aee1

Тези две функции изпълняват дефиницията на изоморфизъм - т.е. имаме следните две равенства:

- `x = hsToUni(uniToHS(x))` за всяка стойност `x: UniversityStudent`, и
- `x = uniToHS(hsToUni(x))` за всяка стойност `x: HighSchoolStudent`.

In [8]:
val alan = HighSchoolStudent(1, "Alan", "Turing")
uniToHS(hsToUni(alan)) == alan

alan: HighSchoolStudent = HighSchoolStudent(1, "Alan", "Turing")
res7_1: Boolean = true

In [9]:
val alonso = HighSchoolStudent(2, "Alonso", "Church")
uniToHS(hsToUni(alonso)) == alonso

alonso: HighSchoolStudent = HighSchoolStudent(2, "Alonso", "Church")
res8_1: Boolean = true

Тъй като всички типове-произведения могат да бъдат репрезентирани от константния полином `1`, всички те са изоморфни.

**Задача**: дефнирайте друг изоморфизъм между `HighSchoolStudent` и `UniversityStudent`.

In [10]:
val hsToUni2: HighSchoolStudent => UniversityStudent = (x: HighSchoolStudent) => ???
val uniToHS2: UniversityStudent => HighSchoolStudent = (x: UniversityStudent) => ???

hsToUni2: HighSchoolStudent => UniversityStudent = ammonite.$sess.cmd9$Helper$$Lambda$2629/0x0000000800d0d840@6dfc5ea0
uniToHS2: UniversityStudent => HighSchoolStudent = ammonite.$sess.cmd9$Helper$$Lambda$2630/0x0000000800d0e840@2acd8042

### Константни Типове

Частен случай на типове-произведение са т.нар. константни типове. Начинът да се дефинира такъв тип в Scala е чрез ключовите думи `case object` или като празен тип-произведение `case class`.

In [11]:
case object Anonymous  // Anonymous singleton object
case class Anonymous() // Anonymous empty case clas (this definition is active for the rest of this document)

defined object Anonymous
defined class Anonymous

Константните типове описват тривиални множества с една единствена стойност.

In [12]:
val anonymous = Anonymous()

anonymous: Anonymous = Anonymous()

Абстрактният тип-израз, с който се обозначава константен тип, е `1`.

Scala предлага предварително дефиниран константен тип `Unit`, чиято единствена стойност се обозначава с `()` или `Unit`.

In [13]:
val u1 = Unit
val u2 = ()

u1 == u2

u1: Unit.type = object scala.Unit
res12_2: Boolean = false

Тъй като всички типове-произведения могат да бъдат репрезентирани от константния полином `1`, всички те са изоморфни.

**Задача**: дефнирайте тривиалния изоморфизъм между `Anonymous` и `Unit`.

In [14]:
val anonymousToUnit: Anonymous => Unit = (a: Anonymous) => ???
val unitToAnonymous: Unit => Anonymous = (u: Unit) => ???

anonymousToUnit: Anonymous => Unit = ammonite.$sess.cmd13$Helper$$Lambda$2664/0x0000000800d22840@44cd6e4c
unitToAnonymous: Unit => Anonymous = ammonite.$sess.cmd13$Helper$$Lambda$2665/0x0000000800d23040@20d7b790

Изборът на нотация тук не е случаен. В унисон с правилата за умножение от първи клас, и при типовете важи равенството

```
1 × Т1 × .. × Тn ≅ Т1 × .. × Тn ≅ Т1 × .. × Тn×1
```


**Задача**: дефнирайте тривиалния изоморфизъм между `HighSchoolStudent` и `HighSchoolStudentExt ≅ Int × String × String × Unit`.

In [15]:
case class HighSchoolStudentExt(socialID: Int, name: String, surname: String, unit: Unit)

defined class HighSchoolStudentExt

In [16]:
val hsStudentToPlus: HighSchoolStudent => HighSchoolStudentExt = (a: HighSchoolStudent) => ???
val plusToHSStudent: HighSchoolStudentExt => HighSchoolStudent = (u: HighSchoolStudentExt) => ???

hsStudentToPlus: HighSchoolStudent => HighSchoolStudentExt = ammonite.$sess.cmd15$Helper$$Lambda$2672/0x0000000800d27840@431c5b79
plusToHSStudent: HighSchoolStudentExt => HighSchoolStudent = ammonite.$sess.cmd15$Helper$$Lambda$2673/0x0000000800d28040@c93ce83

### Типове-Суми (Sum Types)

Докато типовете-произведения ни дават възможност да моделираме цялото като съвкупност от едновременно проявяващи се признаци (т.е. студент с първо име "Alan", фамилия "Turing", студентки номер 1 и т.н.), то типовете-суми ни позволяват да моделираме цялото като съвкупност от взаимно-изключващи се алтернативи.

Абстрактно погледнато, типовете-суми ни позволяват да комбинираме две или повече тип-произведения, използвайки тип-конструктора `·+·`.

Например, възможно е в света, който моделираме, учениците да трябва да събират часове и оценки по определени предмети (задължителни или свободноизбираеми) за всеки срок. Един начин да интегрираме този аспект в алгебричния модел (който за момента се състои единствено от типа `Student`) е чрез нов тип-сума `Subject` и продуктов тип `StudentGrade`.

In [17]:
sealed trait Subject {}
case class MandatorySubject(name: String, term: String, hours: Int) extends Subject
case class OptionalSubject(name: String, term: String, hours: Int) extends Subject
case class SubjectGrade(student: HighSchoolStudent, subject: Subject, grade: Double) {}

defined trait Subject
defined class MandatorySubject
defined class OptionalSubject
defined class SubjectGrade

Този модел ни позволява да изразим факти от типа

- "Алън Туринг слуша Алгебра 1 през пролетта на 2020 година и получи оценка (среден) 3" и
- "Алън Туринг слуша гост-лекцията за функционално програмиране през пролетта на 2020 година и получи оценка 0"

като програмни тойности.

In [18]:
val algebra = MandatorySubject("Algebra 1", "Spring 2020", 3)
val guestLectureFP = OptionalSubject("Functional Programming (Guest Lecture)", "Spring 2020", 2)
val subjectGrade1 = SubjectGrade(alan, algebra, 5.5)
val subjectGrade2 = SubjectGrade(alan, guestLectureFP, 0.0)

algebra: MandatorySubject = MandatorySubject("Algebra 1", "Spring 2020", 3)
guestLectureFP: OptionalSubject = OptionalSubject(
  "Functional Programming (Guest Lecture)",
  "Spring 2020",
  2
)
subjectGrade1: SubjectGrade = SubjectGrade(
  HighSchoolStudent(1, "Alan", "Turing"),
  MandatorySubject("Algebra 1", "Spring 2020", 3),
  5.5
)
subjectGrade2: SubjectGrade = SubjectGrade(
  HighSchoolStudent(1, "Alan", "Turing"),
  OptionalSubject("Functional Programming (Guest Lecture)", "Spring 2020", 2),
  0.0
)

По подобен начин можем да разширим модела на студентите със свободно-избираеми и задължителни предмети.

In [19]:
sealed trait UniversityCourse {}
case class MandatoryCourse(name: String, term: String, credits: Int) extends UniversityCourse
case class OptionalCourse(name: String, term: String, credits: Int) extends UniversityCourse
case class CourseGrade(student: HighSchoolStudent, course: UniversityCourse, grade: Double) {}

defined trait UniversityCourse
defined class MandatoryCourse
defined class OptionalCourse
defined class CourseGrade

In [20]:
val progLang = MandatoryCourse("Programming Language Semantics", "Spring 2020", 3)
val catTheo = OptionalCourse("Category Theory", "Spring 2020", 2)
val grade1 = CourseGrade(alonso, progLang, 6.0)
val grade2 = CourseGrade(alonso, catTheo, 6.0)

progLang: MandatoryCourse = MandatoryCourse(
  "Programming Language Semantics",
  "Spring 2020",
  3
)
catTheo: OptionalCourse = OptionalCourse("Category Theory", "Spring 2020", 2)
grade1: CourseGrade = CourseGrade(
  HighSchoolStudent(2, "Alonso", "Church"),
  MandatoryCourse("Programming Language Semantics", "Spring 2020", 3),
  6.0
)
grade2: CourseGrade = CourseGrade(
  HighSchoolStudent(2, "Alonso", "Church"),
  OptionalCourse("Category Theory", "Spring 2020", 2),
  6.0
)

Тип-изразите на двата модела съответстват.

```
UniversityCourse(...) ≅ MandatoryCourse + OptionalCourse 
                      ≅ String × String × Double + String × String × Double
                      ≅ MandatorySubject + OptionalSubject 
                      ≅ Subject(...)
     
CourseGrade(...) ≅ SubjectGrade(...)
```

Scala ни предоставя възможността да използваме деструктивни шаблони (pattern matching), за да дефинираме функции, които оперират върху тип-сума като съвкупност от случаи - по един за вяко събираемо.

Например, за да извлечем броя задължителни часове, които един предмет допринася към общия брой часове в гимназиалната диплома, можем да използваме следната функция:

In [ ]:
val diplomCredits: Subject => Int = subject => subject match {
    case MandatorySubject(name, term, hours) => hours
    case OptionalSubject(name, term, hours) => 0
}

In [ ]:
diplomCredits(algebra)
diplomCredits(guestLectureFP)

**Задача**: дефинирайте двойка функции, съставляващи изоморфизъм между `Subject` и `UniversityCourse` исползвайки деструктивни шаблони.

## Рекурсивни Типове Данни

Рекурсивните алгебрични типове представляват дефниции, при които името на типа изписан от лявата страна също така фигурира в тип-полинома от дясната. С други думи, поне един от конструкторите на типа референцира себе си в един или повече параметри.

Може би най-простият и интуитивен пример е дефиницията на естествените числа в т.нар единичен код. Неформално, естествените числа могат да бъдат дефинирани по индуктивен начин както следва:

- числото *1* е естествено число,
- ако *x* е естествено число, то и *x + 1*  е естествено число.

Ако изразим горната дефиниция в езика на типовите полиноми, ще получим следното рекурсивно уравнение.

```
Nat ≅ 1 + Nat 
```

Първото събираемо от дясната страна на уравнението кодира константният конструктор *1*, докато второто кодира конструкторът *x + 1*.

Дефиницията може да бъде кодирана в Scala чрез базов тип-сума `Nat` с два варианта `One` (репрезентиращ *1*) и `Succ(x: Nat)` (репрезентиращ *x + 1*). 

In [22]:
sealed trait Nat {}
case class One() extends Nat
case class Succ(x: Nat) extends Nat

defined trait Nat
defined class One
defined class Succ

Ето и първите няколко естествени числа, изразени в единичен код като стойности от тип `Nat`.

In [23]:
val n1 = One()
val n2 = Succ(n1)
val n3 = Succ(n2)
val n4 = Succ(n3)
val n5 = Succ(n4)

n1: One = One()
n2: Succ = Succ(One())
n3: Succ = Succ(Succ(One()))
n4: Succ = Succ(Succ(Succ(One())))
n5: Succ = Succ(Succ(Succ(Succ(One()))))

За да конвертираме от единичен код (`Nat`) в двуичен код `Int`, използваме функция която имплементира т.нар. **структурна рекурсия** върху рекурсивния тип `Nat`. 

In [28]:
val natToInt: Nat => Int = n => n match {
    case One() => 1
    case Succ(x) => natToInt(x) + 1
}

natToInt: Nat => Int = ammonite.$sess.cmd27$Helper$$Lambda$2870/0x0000000800db5840@70c1e8d0

Функцията оперира върху параметър `n` от тип `Nat`, дефинирайки отделен случай за всеки възможен начин, по който `n` би могъл да бъде конструиран.

- Ако `n = One()`, то `natToInt(n) = One()`.
- Ако `n = Succ(x)`, то `natToInt(n) = natToInt(x) + 1`.

**Задача**: имплементирайте функцията `intToNat: Int => Nat` която конвертира двуични `Int` стойности в единичен код.

**Задача**: Разширете модела за студентски лекции така, че курсовете да могат да бъдат организирани в модули. Един модул се състои от един или повече курса или от един или повече под-модули. Използвайки принципа на структурната рекурсия, напишете функции които изчисляват тоталния брой точки (`credits`) в конкретен модул.  